In [1]:
# What version of Python do you have?
import sys
import os
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
import re
import mne
import pathlib
import openpyxl
from datetime import datetime
import pytz
import random
import os
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Sequential
from matplotlib import pyplot as plt
pd.io.parquet.get_engine('auto').__class__
%matplotlib inline

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Init Plugin
Init Graph Optimizer
Init Kernel
Tensor Flow Version: 2.5.0
Keras Version: 2.5.0

Python 3.9.7 (default, Sep 16 2021, 23:53:23) 
[Clang 12.0.0 ]
Pandas 1.3.1
Scikit-Learn 0.24.2
GPU is available


Globals

In [2]:
cwd = os.getcwd()
FREQ = 256
database_path = 'Dataset/CHB-MIT/chb-mit-scalp-eeg-database-1.0.0/'
filtered_database_path = '../Dataset/CHB-MIT/Filtered-chb-mit/'
filted_db_parquet_path = "Dataset/CHB-MIT/dataframe-parquet"
edf_file_type = ".edf"
patient_one_path = 'chb04/'
summary_txt_file_type = "-summary.txt"
external_hardisk_drive_path = os.path.dirname('/Volumes/LaCie/Database/')
cwd

'/Users/niklashjort/Desktop/Notes/Speciale/projects/DataHandling'

Get file paths and info.txt

In [3]:
def get_all_patient_folder_names(database_path):
    folders = os.listdir(database_path)
    patient_folder_names = [(database_path + "/" + x) for x in folders if (x.find(".DS_Store") == -1)]
    return patient_folder_names

def get_all_file_names(directory):
    files = os.listdir(directory)
    edfFileNameList = [(directory + "/" + x) for x in files if (x.endswith(edf_file_type))]
    summary_info_file_name = [(directory + "/" + x) for x in files if (x.endswith(summary_txt_file_type))]
    return (summary_info_file_name[0], edfFileNameList)

Read EDF

In [4]:
def ReadEdfFile(FileName, print_reader_info = False):
    try:
        if(print_reader_info):
            data = mne.io.read_raw_edf(FileName)
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            return converted_raw
        else:
            data = mne.io.read_raw_edf(FileName, verbose='error')
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            return converted_raw
    except Exception as e:
        pass

In [5]:
x = mne.io.read_raw_edf(filtered_database_path + "chb10/chb10_12.edf")
y = ReadEdfFile(filtered_database_path + "chb10/chb10_12.edf")


Extracting EDF parameters from ../Dataset/CHB-MIT/Filtered-chb-mit/chb10/chb10_12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


Save memory

In [6]:
# print(test_df.info(memory_usage='deep'))
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [7]:
def downcast_dtypes(df):
    _start = df.memory_usage(deep=True).sum() / 1024 ** 2
    float_cols = [c for c in df if df[c].dtype == 'float64']
    int_cols = [c for c in df if df[c].dtype in ['int64', 'int32']]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    _end = df.memory_usage(deep=True).sum() / 1024 ** 2
    saved_time = (_start - _end) / _start * 100
    #print(f"Saved: {saved_time:.2f}%")
    return df

format summary txt file

In [8]:
def read_format_info_file(txt_summary_file_path):
    str_container = ""
    with open(txt_summary_file_path, 'r') as f:
        for line in f:
            str_container += str(line).replace("\n", "<br>")

    formatted_str = re.findall('(.*?)<br><br>', str_container)
    bla = [x.group() for x in re.finditer('(.*?)<br><br>', str_container)]
    for index, line in enumerate(formatted_str):
        if re.match('(^Channels in EDF Files:|^Channels changed:)', line):
            formatted_str.remove(formatted_str[index])
        else:
            pass
    return formatted_str

In [9]:
class FileInformationContainer:
    def __init__(self, information_str):
        self.information_str = self.clean_string(information_str)
        self.file_name = self.set_filename()
        self.time_start = self.set_file_time_start_ms()
        self.sz_info = self.set_sz_information()
        
    def clean_string(self, uncleaned_str):
        return uncleaned_str.replace("<br>", " ")

    def set_filename(self):
        filename_found = re.match(r"^File Name: (.+?).edf", self.information_str)
        if filename_found:
            return filename_found.group(1)
        else:
            print(f"{self.file_name} failed get_filename")
            return "filename not found"
    
    def get_milli_sec(self, time_str):
        """Get Seconds from time."""
        h, m, s = time_str.split(':')
        return (int(h) * 3600 + int(m) * 60 + int(s)) * 1000

    def set_file_time_start_ms(self):
        time_start_found = re.match(r".*File Start Time: (.*?) File", self.information_str)
        if time_start_found:
            try:
                return self.get_milli_sec(time_start_found.group(1))
            except Exception as e :
                print(f"{self.file_name}: error {e} cannot convert to ms time")
                return f"{e}"
        else:
            print(f"{self.file_name} failed get_file_time_start_ms")
            return "time start not found"
    
    def get_sz_count(self):
        sz_count = 0
        count_found = re.search(r".*Seizures in File: (.*?) Seizure", self.information_str)
        if count_found:
            sz_count = count_found.group(1)
        if int(sz_count) > 0:
            return int(sz_count)
        else:
            return 0
        

    def set_sz_information(self):
        if(type(self.get_sz_count()) != None and self.get_sz_count() > 0):
            try:
                pattern = re.compile(r"Seizure [1-9] (?P<state>[?:Start|End]+) Time: (?P<Sec>[0-9]+)")
                myList = [m.groupdict() for m in pattern.finditer(self.information_str)]
                if(len(myList) <= 0):
                     pattern = re.compile(r"Seizure (?P<state>[?:Start|End]+) Time: (?P<Sec>[0-9]+)")
                     myList = [m.groupdict() for m in pattern.finditer(self.information_str)]
                for item in myList:
                    converted_time = int(item.get("Sec")) * 1000
                    item["Sec"] = converted_time
                formatted = []
                for i in range(0, len(myList), 2):
                    formatted.append({"sz_start" : myList[i]["Sec"], "sz_end" : myList[i+1]["Sec"]})
                return formatted
            except Exception as e:
                print(f"set_sz_information failed at file: {self.file_name} with the following exception: {e}")
        else:
            return []

    def get(self):
        return self.information_str

Timestamp inserts

In [10]:
def insert_time_stamp(dataframe, file_start_time):
    period_row_increment_value =  (1 / 256) * 1000
    dataframe.insert(0, "timestamp", [file_start_time + i * period_row_increment_value for i in dataframe.index])

Insert classes

In [49]:
def insert_class_col(dataframe, sz_info_list):
    
    if "class" not in dataframe.columns:
        dataframe.insert(0, "class", np.nan)

    if len(sz_info_list) == 0:
        dataframe.loc[(dataframe['class'] != "seizure") & (dataframe['class'] != "Preictal I") & (dataframe['class'] != "Preictal II"), "class"] = "Interictal"
    else:
        for item in sz_info_list:
            sz_start = item["sz_start"]
            sz_end = item["sz_end"]
            #print(f"hihi: {dataframe.iloc[0]['timestamp']}")
            actual_sz_start = dataframe.iloc[0]['timestamp'] + sz_start
            preictal_one_start = actual_sz_start - (30 * 60 * 1000)
            preictal_two_start = actual_sz_start - (10 * 60 * 1000)
            actual_sz_end = dataframe.iloc[0]['timestamp'] + sz_end
            dataframe['timestamp'] = pd.to_numeric(dataframe['timestamp'])

            # #INSERTING INTERICTAL
            # dataframe.loc[(dataframe['class'] != "seizure") | (dataframe['class'] != "Preictal I") | (dataframe['class'] != "Preictal II") | (dataframe['timestamp'] > actual_sz_end) | (dataframe['timestamp'] < actual_sz_start), "class"] = "Interictal"
            dataframe.loc[(dataframe['class'] != "seizure") & (dataframe['class'] != "Preictal II") & (dataframe['class'] != "Preictal I"), "class"] = "Interictal"

            #INSERTING PREICTAL I
            dataframe.loc[(dataframe['class'] != "seizure") & (dataframe['class'] != "Preictal II")  & (dataframe['timestamp'] >= preictal_one_start) & (dataframe['timestamp'] < preictal_two_start), "class"] = "Preictal I"

            #INSERTING PREICTAL II
            dataframe.loc[(dataframe['class'] != "seizure") & (dataframe['timestamp'] >= preictal_two_start) & (dataframe['timestamp'] < actual_sz_start), "class"] = "Preictal II"

            #INSERTING SEIZURE CLASS
            dataframe.loc[(dataframe['timestamp'] >= actual_sz_start) & (dataframe['timestamp'] < actual_sz_end), "class"] = "seizure"

            #print(dataframe["class"].value_counts())

Compress and save

In [55]:
def df_save_compress(filename, df):
    df.to_parquet(f"{external_hardisk_drive_path}/Parquet_Seizure/{filename}.parquet.gzip", compression="gzip")

Run script

In [56]:
for patient in get_all_patient_folder_names(filtered_database_path):
    print(patient)
    current_patient = patient
    info_txt_path, edf_files = get_all_file_names(current_patient)
    # read & extract information
    info_txt = read_format_info_file(info_txt_path)
    for line in info_txt[1:]:
        #print(line)
        edf_info_container = FileInformationContainer(line)
        selected_edf_path = [x for x in edf_files if (edf_info_container.file_name in x)][0]
        edf_df = ReadEdfFile(selected_edf_path)
        if edf_info_container.get_sz_count() > 0:
            if edf_df is not None:
                edf_df = downcast_dtypes(edf_df)
                insert_time_stamp(edf_df, edf_info_container.time_start)
                #print(f"info list = {edf_info_container.sz_info}")
                insert_class_col(edf_df, edf_info_container.sz_info)
                print(f"filename: {edf_info_container.file_name} classes: {edf_df['class'].value_counts()}")
                print("-------------------------------------------------------")
                print(edf_info_container.file_name)
                df_save_compress(edf_info_container.file_name, edf_df)

../Dataset/CHB-MIT/Filtered-chb-mit//chb20
filename: chb20_12 classes: Interictal     890112
Preictal II     24064
seizure          7424
Name: class, dtype: int64
-------------------------------------------------------
chb20_12
filename: chb20_13 classes: Preictal I     324608
Preictal II    307200
Interictal     272128
seizure         17664
Name: class, dtype: int64
-------------------------------------------------------
chb20_13
filename: chb20_14 classes: Interictal     451072
Preictal I     307200
Preictal II    153600
seizure          9728
Name: class, dtype: int64
-------------------------------------------------------
chb20_14
filename: chb20_15 classes: Interictal     476672
Preictal II    253440
Preictal I     169984
seizure         21504
Name: class, dtype: int64
-------------------------------------------------------
chb20_15
filename: chb20_16 classes: Interictal     418304
Preictal I     307200
Preictal II    153600
seizure          8960
Name: class, dtype: int64
---------

In [44]:
x = ReadEdfFile("../Dataset/CHB-MIT/Filtered-chb-mit//chb20//chb20_13.edf")
info_txt = read_format_info_file("../Dataset/CHB-MIT/Filtered-chb-mit//chb20//chb20-summary.txt")[11:12]
print(info_txt)
ex_con = FileInformationContainer(info_txt[0])
print("2")
print(ex_con.get_sz_count())
insert_time_stamp(x, ex_con.time_start)
print(f"info list = {edf_info_container.sz_info}")
insert_class_col(x, ex_con.sz_info)


x.isna().sum()

['File Name: chb20_13.edf<br>File Start Time: 04:57:00<br>File End Time: 5:57:00<br>Number of Seizures in File: 2<br>Seizure 1 Start Time: 1440 seconds<br>Seizure 1 End Time: 1470 seconds<br>Seizure 2 Start Time: 2498 seconds<br>Seizure 2 End Time: 2537 seconds']
2
2
info list = []
Interictal     545280
Preictal I     215040
Preictal II    153600
seizure          7680
Name: class, dtype: int64
Preictal I     324608
Preictal II    307200
Interictal     272128
seizure         17664
Name: class, dtype: int64


class        0
timestamp    0
FP1-F7       0
F7-T7        0
T7-P7        0
P7-O1        0
.-0          0
FP1-F3       0
F3-C3        0
C3-P3        0
P3-O1        0
.-1          0
FZ-CZ        0
CZ-PZ        0
.-2          0
FP2-F4       0
F4-C4        0
C4-P4        0
P4-O2        0
.-3          0
FP2-F8       0
F8-T8        0
T8-P8        0
P8-O2        0
.-4          0
P7-T7        0
T7-FT9       0
FT9-FT10     0
FT10-T8      0
STI 014      0
dtype: int64